In [ ]:
import pandas as pd

#Specify target location
target_location =input("enter location : ")

# Specify the file path
hotels_file_path = r'/content/preprocessed_hotels.csv'
# Specify the file path
restuarants_file_path = r'/content/Restuarant_EDITED.xlsx'


def get_top_hotels(hotels_file_path, target_location, top_n=15):
    try:
        # Load the restaurant data from a CSV file
        df = pd.read_csv(hotels_file_path)

        # Filter hotels based on the specified location
        filtered_hotels = df[df['Area'] == target_location]

        # Sort hotels by ratings (assuming ratings are in a column named 'Rating')
        sorted_hotels = filtered_hotels.sort_values(by='Rating', ascending=False)

        # Get the top hotels
        top_hotels = sorted_hotels.head(top_n)

        return top_hotels
    except FileNotFoundError:
        print(f"File '{hotels_file_path}' not found.")
        return None
    except PermissionError:
        print(f"Permission denied to access file '{hotels_file_path}'. Please check permissions.")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None



def get_top_restaurants(restuarants_file_path, target_location, top_n=15):
    try:
        # Load the restaurant data from an Excel file
        df = pd.read_excel(restuarants_file_path)

        # Filter restaurants based on the specified location
        filtered_restaurants = df[df['Location'] == target_location]

        # Sort restaurants by ratings (assuming ratings are in a column named 'Rating')
        sorted_restaurants = filtered_restaurants.sort_values(by='Rating', ascending=False)

        # Get the top restaurants
        top_restaurants = sorted_restaurants.head(top_n)

        return top_restaurants
    except FileNotFoundError:
        print(f"File '{restuarants_file_path}' not found.")
        return None
    except PermissionError:
        print(f"Permission denied to access file '{restuarants_file_path}'. Please check permissions.")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None




# Get the top hotels for the specified location
top_hotels = get_top_hotels(hotels_file_path, target_location)


# Get the top restaurants for the specified location
top_restaurants = get_top_restaurants(restuarants_file_path, target_location)



# Print the names and ratings of the top hotels
if top_hotels is not None:
    for index, row in top_hotels.iterrows():
        print(f"{row['Hotel_Name']}: {row['Rating']}")

print('------------------------------------------------------------------------------------------------------')


# Print the names and ratings of the top restaurants
if top_restaurants is not None:
    for index, row in top_restaurants.iterrows():
        print(f"{row['Restaurant_Name']}: {row['Rating']}")

Taj Krishna, Hyderabad: 4.5
Taj Deccan, Hyderabad: 4.4
Super OYO Collection O Rcc Hotel Banjara Hills: 4.4
Destin Olympus: 4.3
URBAN OASIS: 4.3
Kygo Hostels: 4.2
Hyatt Place Hyderabad Banjara Hills: 4.1
Lemon Tree Hotel, Banjara Hills, Hyderabad: 4.1
BURWOOD SUITES: 4.0
Beehive Commune: 4.0
Minerva Grand - Banjara: 4.0
Manira Service Apartments: 3.9
The Landmark Banjara Hills: 3.8
OYO FM GUEST HOUSE: 3.654308617
FabExpress Luxury Inn: 3.654308617
------------------------------------------------------------------------------------------------------
Varalakshmi Tiffins: 4.5
Flavours of Taj Mahal Hotel: 4.4
Chili's American Grill & Bar: 4.3
Huber & Holly: 4.1


In [ ]:
#################################################
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load user's visited places data from Excel
user_visited_df = pd.read_excel('/user_history.xlsx')

# Load list of all places data from Excel
all_places_df = pd.read_excel('/locations_with_desc.xlsx')

# Fill missing values in the 'DESCRIPTION' column with empty strings
all_places_df['DESCRIPTION'].fillna('', inplace=True)

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform descriptions of all places to TF-IDF vectors
all_places_tfidf = tfidf_vectorizer.fit_transform(all_places_df['DESCRIPTION'])

# Fit and transform descriptions of visited places to TF-IDF vectors
user_visited_tfidf = tfidf_vectorizer.transform(user_visited_df['Description'])

# Compute cosine similarity between visited places and all places
cosine_sim = cosine_similarity(user_visited_tfidf, all_places_tfidf)


def recommend_places(user_visited_df, all_places_df, cosine_sim, top_n=5):
    recommendations = []

    for i, visited_row in user_visited_df.iterrows():
        visited_place = visited_row['Visited_Places']
        visited_category = visited_row['V_Themes']
        indices = cosine_sim[i].argsort()[-top_n:][::-1]

        for index in indices:
            place_info = all_places_df.iloc[index]
            place = place_info['Name Of The Place']
            category = place_info['Themes']

            # Filter out recommendations that are not contextually relevant
            if category == visited_category:
                location_area = place_info['Address']
                recommendations.append({'Visited Place': visited_place,
                                        'Recommended Place': place,
                                        'Category': category,
                                        'Location/Area': location_area})

    return pd.DataFrame(recommendations)

# Recommend places to the user
recommended_places = recommend_places(user_visited_df, all_places_df, cosine_sim)

# Display and store the recommended places
print(recommended_places)
recommended_places.to_excel('recommended_places.xlsx', index=False)

# Let the user select a place
selected_place = input("Select a place from the recommendations: ")

# Find the corresponding location
selected_place_location = recommended_places[recommended_places['Recommended Place'] == selected_place]['Location/Area'].values[0]

print(f"The location of {selected_place} is: {selected_place_location}")

# Specify target location
target_location = selected_place_location

# Specify the file paths
hotels_file_path = '/content/preprocessed_hotels.csv'
restaurants_file_path = '/content/Restuarant_EDITED.xlsx'

# Define functions to retrieve top hotels and restaurants
def get_top_hotels(hotels_file_path, target_location, top_n=15):
    try:
        # Load hotel data from CSV file
        df = pd.read_csv(hotels_file_path)

        # Filter hotels by target location
        filtered_hotels = df[df['Area'] == target_location]

        # Sort hotels by ratings
        sorted_hotels = filtered_hotels.sort_values(by='Rating', ascending=False)

        # Get top hotels
        top_hotels = sorted_hotels.head(top_n)

        return top_hotels
    except Exception as e:
        print(f"An error occurred while retrieving top hotels: {str(e)}")
        return None

def get_top_restaurants(restaurants_file_path, target_location, top_n=15):
    try:
        # Load restaurant data from Excel file
        df = pd.read_excel(restaurants_file_path)

        # Filter restaurants by target location
        filtered_restaurants = df[df['Location'] == target_location]

        # Sort restaurants by ratings
        sorted_restaurants = filtered_restaurants.sort_values(by='Rating', ascending=False)

        # Get top restaurants
        top_restaurants = sorted_restaurants.head(top_n)

        return top_restaurants
    except Exception as e:
        print(f"An error occurred while retrieving top restaurants: {str(e)}")
        return None

# Get and display top hotels
print("Top Hotels:")
top_hotels = get_top_hotels(hotels_file_path, target_location)
if top_hotels is not None:
    print(top_hotels[['Hotel_Name', 'Rating']])

print('------------------------------------------------------------------------------------------------------')

# Get and display top restaurants
print("Top Restaurants:")
top_restaurants = get_top_restaurants(restaurants_file_path, target_location)
if top_restaurants is not None:
    print(top_restaurants[['Restaurant_Name', 'Rating']])


            Visited Place                                 Recommended Place  \
0        Ramoji Film City                                  Ramoji Film City   
1           Golconda Fort                                     Golconda Fort   
2           Golconda Fort                                   Rachakonda Fort   
3      Wonderla Hyderabad                                Wonderla Hyderabad   
4      Wonderla Hyderabad                                       Thrill City   
5      Wonderla Hyderabad               Mount Opera Multi-Theme Park Resort   
6            Birla Mandir                                      Birla Mandir   
7            Birla Mandir                    Sri Venkateshwara Swamy Temple   
8            Birla Mandir                                      Shyam Mandir   
9            Birla Mandir  ISKCON Hyderabad Sri Sri Radha Madanmohan Mandir   
10      Salar Jung Museum                                 Salar Jung Museum   
11   Taj Falaknuma Palace                           